In [1]:
%matplotlib widget

In [2]:
# Imports
import sys
import os 
import numpy as np
import pandas as pd
from astropy.visualization import hist
from tqdm import tqdm

from spaxelsleuth.loaddata.lzifu import load_lzifu_galaxies
from spaxelsleuth.loaddata.sami import load_sami_galaxies
from spaxelsleuth.plotting.plottools import plot_empty_BPT_diagram
from spaxelsleuth.plotting.plottools import vmin_fn, vmax_fn, label_fn, cmap_fn, fname_fn
from spaxelsleuth.plotting.plottools import bpt_colours, bpt_labels, whav_colors, whav_labels
from spaxelsleuth.plotting.plottools import morph_labels, morph_ticks
from spaxelsleuth.plotting.plottools import ncomponents_labels, ncomponents_colours
from spaxelsleuth.plotting.plottools import component_labels, component_colours
from spaxelsleuth.plotting.plotgalaxies import plot2dhistcontours, plot2dscatter, plot2dcontours
from spaxelsleuth.plotting.plot2dmap import plot2dmap
from spaxelsleuth.plotting.sdssimg import plot_sdss_image

import matplotlib
from matplotlib import rc, rcParams
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from IPython.core.debugger import Tracer

rc("text", usetex=False)
rc("font",**{"family": "serif", "size": 14})
rcParams["savefig.bbox"] = "tight"
rcParams["savefig.format"] = "pdf"
plt.ion()
plt.close("all")


In [3]:
# Options
fig_path = "/priv/meggs3/u5708159/SAMI/figs/paper/"
savefigs = False
bin_type = "default"    # Options: "default" or "adaptive" for Voronoi binning
ncomponents = "recom"   # Options: "1" or "recom"
eline_SNR_min = 3       # Minimum S/N of emission lines to accept
plt.close("all")


# TEST: S/N in emission line fits 
---
* 3 emission line components w/ fixed S/N ratio each
* use mpfit to fit the lines 
* what is the formal S/N on the fitted emission lines?


In [13]:
from mpfit import mpfit

In [14]:
def gaussian(x, A, mu, sigma):
    return A * np.exp(-(x - mu)**2 / (2 * sigma**2))



In [110]:
# Create our emission lines 
x_vals = np.linspace(-500, 500, 200)

# Component 1
A_1 = 10
mu_1 = 0
sigma_1 = 40
f_1 = gaussian(x_vals, A_1, mu_1, sigma_1)

# Component 2
A_2 = 5
mu_2 = -10
sigma_2 = 100
f_2 = gaussian(x_vals, A_2, mu_2, sigma_2)

# Component 3
A_3 = 3 
mu_3 = -20
sigma_3 = 150
f_3 = gaussian(x_vals, A_3, mu_3, sigma_3)

spec_1comp_no_noise = f_1
spec_2comp_no_noise = f_1 + f_2 
spec_3comp_no_noise = f_1 + f_2 + f_3

# # Inspect result
# fig, ax = plt.subplots(1, 1)
# ax.plot(x_vals, f_1, label="Component 1", linestyle=":")
# ax.plot(x_vals, f_2, label="Component 2", linestyle=":")
# ax.plot(x_vals, f_3, label="Component 3", linestyle=":")
# ax.plot(x_vals, spec_1comp_no_noise, label="1 components", linestyle="-", linewidth=0.5)
# ax.plot(x_vals, spec_2comp_no_noise, label="2 components", linestyle="-", linewidth=0.5)
# ax.plot(x_vals, spec_3comp_no_noise, label="3 components", linestyle="-", linewidth=0.5)
# ax.plot(x_vals, spec_1comp, label="1 components (noise)", linestyle="-")
# ax.plot(x_vals, spec_2comp, label="2 components (noise)", linestyle="-")
# ax.plot(x_vals, spec_3comp, label="3 components (noise)", linestyle="-")
# ax.legend()


In [103]:
################################################################################
# Fit using mpfit 
################################################################################
# The model we fit to the data
def F(x, p):
    A_1, mu_1, sigma_1, A_2, mu_2, sigma_2, A_3, mu_3, sigma_3 = p
    return gaussian(x, A_1, mu_1, sigma_1) + gaussian(x, A_2, mu_2, sigma_2) + gaussian(x, A_3, mu_3, sigma_3)

############################################################################
# Function to be passed into mpfit
def minfunc(p, fjac=None, x=None, y=None, err=None):
    # The model evaluated at the provided x values with parameters stored in p
    model = F(x, p)
    # The deviates are the differences between the measurements y and the model
    # evaluated at each point x normalised by the standard deviation in the
    # measurement.
    deviates = np.array((y - model) / err, dtype=float)
    # We return p status flag and the deviates.
    return 0, deviates

############################################################################
# Parameters to be passed into the fitting function
parinfo = [{
    'value': 0,
    'fixed': False,
    'limited': [False, False],
    'limits':[0, 0],
    'parname':'',
    'mpside':2,
    'mpprint':0
} for ii in range(3 * 3)]
parinfo[0]['parname'] = 'A_1'
parinfo[1]['parname'] = 'mu_1'
parinfo[2]['parname'] = 'sigma_1'
parinfo[3]['parname'] = 'A_2'
parinfo[4]['parname'] = 'mu_2'
parinfo[5]['parname'] = 'sigma_2'
parinfo[6]['parname'] = 'A_3'
parinfo[7]['parname'] = 'mu_3'
parinfo[8]['parname'] = 'sigma_3'

############################################################################
# Parameter constraints
# Amplitude: must be positive
for ii in [0, 3, 6]:
    parinfo[ii]['limited'] = [True, True]
    parinfo[ii]['limits'] = [0, 100.]

# Mean: constrained to be inside the wavelength window
for ii in [1, 4, 7]:
    parinfo[3]['limited'] = [True, True]
    parinfo[4]['limits'] = [-500., +500.]

# Sigma
for ii in [2, 5, 8]:
    parinfo[ii]['limited'] = [True, True]
parinfo[2]['limits'] = [0, 80.]
parinfo[5]['limits'] = [80, 200.]
parinfo[8]['limits'] = [200, 250.]

In [117]:
################################################################################
# Run the fit: 1 component
################################################################################
SN_comp_1_vals_1comp = []
SN_comp_2_vals_1comp = []
SN_comp_3_vals_1comp = []
niters = 1000
for nn in tqdm(range(niters)):
    # Add random noise
    noise_stdv = 0.1
    spec_err = np.full_like(x_vals, noise_stdv)
    spec_1comp = spec_1comp_no_noise + np.random.normal(loc=0, scale=spec_err)
    parnames = {
        'x': x_vals,
        'y': spec_1comp,
        'err': spec_err
    }

    # Run mpfit
    p0 = np.array([1., 0., 80.] + [0., 0., 100.] + [0., 0., 200.])
    fit = mpfit.mpfit(minfunc, p0, functkw=parnames, parinfo=parinfo, quiet=1)
    p_fit = fit.params
    p_err = fit.perror
    spec_fit = F(x_vals, p_fit)

    if nn == 0:
        # Inspect result
        fig, ax = plt.subplots(1, 1)
        ax.plot(x_vals, f_1, label="Component 1")
        ax.plot(x_vals, f_2, label="Component 2")
        ax.plot(x_vals, f_3, label="Component 3")
        ax.plot(x_vals, parnames["y"], label="Spectrum")
        ax.plot(x_vals, spec_fit, "k", label="Fit")
        ax.legend()

    # Compute S/N in the fitted fluxes
    A_1_fit, mu_1_fit, sigma_1_fit, A_2_fit, mu_2_fit, sigma_2_fit, A_3_fit, mu_3_fit, sigma_3_fit = p_fit
    A_1_err, mu_1_err, sigma_1_err, A_2_err, mu_2_err, sigma_2_err, A_3_err, mu_3_err, sigma_3_err = p_err
    flux_1_fit = np.sqrt(2 * np.pi) * A_1_fit * sigma_1_fit
    flux_1_fit_err = np.sqrt(2 * np.pi * (A_1**2 * sigma_1_err**2 + sigma_1**2 * A_1_err**2) )
    flux_2_fit = np.sqrt(2 * np.pi) * A_2_fit * sigma_2_fit
    flux_2_fit_err = np.sqrt(2 * np.pi * (A_2**2 * sigma_2_err**2 + sigma_2**2 * A_2_err**2) )
    flux_3_fit = np.sqrt(2 * np.pi) * A_3_fit * sigma_3_fit
    flux_3_fit_err = np.sqrt(2 * np.pi * (A_3**2 * sigma_3_err**2 + sigma_3**2 * A_3_err**2) )

    # Save the S/N
    SN_comp_1_vals_1comp.append(flux_1_fit / flux_1_fit_err)
    SN_comp_2_vals_1comp.append(flux_2_fit / flux_2_fit_err)
    SN_comp_3_vals_1comp.append(flux_3_fit / flux_3_fit_err)
    

  0%|          | 0/1000 [00:00<?, ?it/s]/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

  0%|          | 1/1000 [00:00<01:52,  8.86it/s]/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in double_scalars
/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in double_scalars
100%|██████████| 1000/1000 [01:07<00:00, 14.81it/s]


In [118]:
################################################################################
# Run the fit: 2 components
################################################################################
SN_comp_1_vals_2comp = []
SN_comp_2_vals_2comp = []
SN_comp_3_vals_2comp = []
niters = 1000
for nn in tqdm(range(niters)):
    # Add random noise
    noise_stdv = 0.1
    spec_err = np.full_like(x_vals, noise_stdv)
    spec_2comp = spec_2comp_no_noise + np.random.normal(loc=0, scale=spec_err)
    parnames = {
        'x': x_vals,
        'y': spec_2comp,
        'err': spec_err
    }

    # Run mpfit
    p0 = np.array([1., 0., 80.] + [0., 0., 100.] + [0., 0., 200.])
    fit = mpfit.mpfit(minfunc, p0, functkw=parnames, parinfo=parinfo, quiet=1)
    p_fit = fit.params
    p_err = fit.perror

    if nn == 0:
        # Inspect result
        fig, ax = plt.subplots(1, 1)
        spec_fit = F(x_vals, p_fit)
        ax.plot(x_vals, f_1, label="Component 1")
        ax.plot(x_vals, f_2, label="Component 2")
        ax.plot(x_vals, f_3, label="Component 3")
        ax.plot(x_vals, parnames["y"], label="Spectrum")
        ax.plot(x_vals, spec_fit, "k", label="Fit")
        ax.legend()

    # Compute S/N in the fitted fluxes
    A_1_fit, mu_1_fit, sigma_1_fit, A_2_fit, mu_2_fit, sigma_2_fit, A_3_fit, mu_3_fit, sigma_3_fit = p_fit
    A_1_err, mu_1_err, sigma_1_err, A_2_err, mu_2_err, sigma_2_err, A_3_err, mu_3_err, sigma_3_err = p_err
    flux_1_fit = np.sqrt(2 * np.pi) * A_1_fit * sigma_1_fit
    flux_1_fit_err = np.sqrt(2 * np.pi * (A_1**2 * sigma_1_err**2 + sigma_1**2 * A_1_err**2) )
    flux_2_fit = np.sqrt(2 * np.pi) * A_2_fit * sigma_2_fit
    flux_2_fit_err = np.sqrt(2 * np.pi * (A_2**2 * sigma_2_err**2 + sigma_2**2 * A_2_err**2) )
    flux_3_fit = np.sqrt(2 * np.pi) * A_3_fit * sigma_3_fit
    flux_3_fit_err = np.sqrt(2 * np.pi * (A_3**2 * sigma_3_err**2 + sigma_3**2 * A_3_err**2) )

    # Save the S/N
    SN_comp_1_vals_2comp.append(flux_1_fit / flux_1_fit_err)
    SN_comp_2_vals_2comp.append(flux_2_fit / flux_2_fit_err)
    SN_comp_3_vals_2comp.append(flux_3_fit / flux_3_fit_err)

  0%|          | 0/1000 [00:00<?, ?it/s]/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in double_scalars
100%|██████████| 1000/1000 [01:07<00:00, 14.77it/s]


In [122]:
################################################################################
# Run the fit: 3 components
################################################################################
SN_comp_1_vals_2comp = []
SN_comp_2_vals_2comp = []
SN_comp_3_vals_2comp = []
niters = 1000
for nn in tqdm(range(niters)):
    # Add random noise
    noise_stdv = 0.1
    spec_err = np.full_like(x_vals, noise_stdv)
    spec_3comp = spec_3comp_no_noise + np.random.normal(loc=0, scale=spec_err)
    parnames = {
        'x': x_vals,
        'y': spec_3comp,
        'err': spec_err
    }

    # Run mpfit
    p0 = np.array([1., 0., 80.] + [0., 0., 100.] + [0., 0., 200.])
    fit = mpfit.mpfit(minfunc, p0, functkw=parnames, parinfo=parinfo, quiet=1)
    p_fit = fit.params
    p_err = fit.perror

    if nn == 0:
        # Inspect result
        fig, ax = plt.subplots(1, 1)
        spec_fit = F(x_vals, p_fit)
        ax.plot(x_vals, f_1, label="Component 1")
        ax.plot(x_vals, f_2, label="Component 2")
        ax.plot(x_vals, f_3, label="Component 3")
        ax.plot(x_vals, parnames["y"], label="Spectrum")
        ax.plot(x_vals, spec_fit, "k", label="Fit")
        ax.legend()

    # Compute S/N in the fitted fluxes
    A_1_fit, mu_1_fit, sigma_1_fit, A_2_fit, mu_2_fit, sigma_2_fit, A_3_fit, mu_3_fit, sigma_3_fit = p_fit
    A_1_err, mu_1_err, sigma_1_err, A_2_err, mu_2_err, sigma_2_err, A_3_err, mu_3_err, sigma_3_err = p_err
    flux_1_fit = np.sqrt(2 * np.pi) * A_1_fit * sigma_1_fit
    flux_1_fit_err = np.sqrt(2 * np.pi * (A_1**2 * sigma_1_err**2 + sigma_1**2 * A_1_err**2) )
    flux_2_fit = np.sqrt(2 * np.pi) * A_2_fit * sigma_2_fit
    flux_2_fit_err = np.sqrt(2 * np.pi * (A_2**2 * sigma_2_err**2 + sigma_2**2 * A_2_err**2) )
    flux_3_fit = np.sqrt(2 * np.pi) * A_3_fit * sigma_3_fit
    flux_3_fit_err = np.sqrt(2 * np.pi * (A_3**2 * sigma_3_err**2 + sigma_3**2 * A_3_err**2) )

    # Save the S/N
    SN_comp_1_vals_2comp.append(flux_1_fit / flux_1_fit_err)
    SN_comp_2_vals_2comp.append(flux_2_fit / flux_2_fit_err)
    SN_comp_3_vals_2comp.append(flux_3_fit / flux_3_fit_err)

  0%|          | 0/1000 [00:00<?, ?it/s]/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

100%|██████████| 1000/1000 [01:05<00:00, 15.18it/s]


In [123]:
# Plot histograms showing the S/N in component 1 for the 1-component spectrum and the 2-component spectrum
fig, ax = plt.subplots(1, 1)
hist(np.array(SN_comp_1_vals_1comp), bins=50, range=(0, 300), label="1-component spectrum", histtype="step")
hist(np.array(SN_comp_1_vals_2comp), bins=50, range=(0, 300), label="2-component spectrum", histtype="step")
hist(np.array(SN_comp_1_vals_3comp), bins=50, range=(0, 300), label="3-component spectrum", histtype="step")
ax.legend()
ax.set_xlabel("S/N in component 1")

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'SN_comp_1_vals_3comp' is not defined

# SAMI dataset 
---

In [52]:
# Load the sample
df = load_sami_galaxies(ncomponents=ncomponents,
                        bin_type=bin_type,
                        eline_SNR_min=eline_SNR_min, 
                        vgrad_cut=False,
                        correct_extinction=False,
                        sigma_gas_SNR_cut=True)

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc,

In [81]:
# Load the sample
df_1comp = load_sami_galaxies(ncomponents='1',
                              bin_type=bin_type,
                              eline_SNR_min=eline_SNR_min, 
                              vgrad_cut=False,
                              correct_extinction=False,
                              sigma_gas_SNR_cut=True)

In [82]:
for col_z in ["count", "Number of components", "BPT (numeric) (total)"]:
    fig, axs, cax = plot_empty_BPT_diagram(colorbar=True, nrows=1, include_Law2021=True)
    
    # Plot 2D histograms of the subset
    plot2dhistcontours(df_1comp, col_x="log N2 (total)", col_y="log O3 (total)", col_z=col_z, log_z=True if col_z == "count" else False, vmin=1 if col_z == "count" else None, vmax=1e3 if col_z == "count" else None, ax=axs[0], nbins=100, contours=True, colors="white", plot_colorbar=False)
    plot2dhistcontours(df_1comp, col_x="log S2 (total)", col_y="log O3 (total)", col_z=col_z, log_z=True if col_z == "count" else False, vmin=1 if col_z == "count" else None, vmax=1e3 if col_z == "count" else None, ax=axs[1], nbins=100, contours=True, colors="white", plot_colorbar=False)
    plot2dhistcontours(df_1comp, col_x="log O1 (total)", col_y="log O3 (total)", col_z=col_z, log_z=True if col_z == "count" else False, vmin=1 if col_z == "count" else None, vmax=1e3 if col_z == "count" else None, ax=axs[2], nbins=100, contours=True, colors="white", cax=cax, plot_colorbar=True)

    # Decorations
    [ax.set_ylabel("") for ax in axs[1:]]

    # Grid on
    [ax.grid() for ax in axs]
    
    # Save
    if savefigs:
        fname = os.path.join(fig_path, f"BPT_SAMI_{fname_fn(col_z)}_1comp.pdf")
        fig.savefig(fname, bbox_inches="tight", format="pdf")
        print(f"File saved at: {fname}")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## We know that the fraction of spaxels with multiple/KD components increases as a function of SFR/SFR surface density. But at a fixed SFR/SFR surface density, only *some* spaxels have KD/multiple kinematic components. **Why is this?** What makes the spaxels with KD/multiple components *different* from those without?
---

In [53]:
# Make a separate DataFrame only containing star-forming spaxels
df_SF = df.copy()
df_SF = df_SF[df_SF["BPT (total)"] == "SF"]

# New columns: total number of kinematically disturbed components
df_SF["Number of kinematically disturbed components"] = df_SF["Kinematically disturbed (component 0)"].astype(int) + df_SF["Kinematically disturbed (component 1)"].astype(int) + df_SF["Kinematically disturbed (component 2)"].astype(int)


In [54]:
# Pick spaxels within a narrow range of SFR surface density
SFR_lower = -1.5
SFR_upper = -1.25
df_SF_subset = df_SF[(df_SF["log SFR surface density (component 0)"] > SFR_lower) & (df_SF["log SFR surface density (component 0)"] <= SFR_upper)]
print(f"N = {df_SF_subset.shape[0]}")

N = 11833


In [77]:
# Halpha EW, SFR, SFR surface density
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)

for cc, col_x in enumerate(["log HALPHA EW (total)", "log SFR (component 0)", "log SFR surface density (component 0)"]):
    for ii in range(3):
        cond = df_SF_subset["Number of components"] == ii + 1
        hist(df_SF_subset.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc], range=(vmin_fn(col_x), 100 if col_x.startswith("sigma") else vmax_fn(col_x)),
             bins="scott",
             label=f"{ncomponents_labels[ii]} KD component{'s' if ii  > 1 else ''}" + r" ($N = %d$)" % (df_SF_subset.loc[cond, col_x].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if ii > 0 else None
axs[0].axvline(np.log10(3), linestyle="--", color="k")
axs[0].legend(fontsize="x-small", loc="upper left")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle(r"H$\alpha$-derived quantities (SF spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_KDcomponents_HaEW_and_sigma_and_SFR_SF_only")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight")

# Systematic quantities
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)
for cc, col_x in enumerate(["Bin size (square kpc)", "z_spec", "Inclination i (degrees)"]):
    for ii in range(3):
        cond = df_SF_subset["Number of components"] == ii + 1
        hist(df_SF_subset.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc],
             bins="scott", 
             range=(0, 90) if col_x == "Inclination i (degrees)" else None,
             label=f"{ncomponents_labels[ii]} KD component{'s' if ii > 1 else ''}" + r" ($N = %d$)" % (df_SF_subset.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if cc > 0 else None
# axs[0].legend(fontsize="x-small", loc="upper right")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle("Systematics (SF spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_KDcomponents_systematic_SF_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

# Systematic quantities
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(20, 4))
fig.subplots_adjust(wspace=0)
for cc, col_x in enumerate(["HALPHA S/N (total)", "HALPHA S/N (component 0)", "HALPHA S/N (component 1)", "HALPHA S/N (component 2)"]):
    for ii in range(3):
        cond = df_SF_subset["Number of components"] == ii + 1
        hist(df_SF_subset.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc],
             bins="scott", 
             range=(0, 90) if col_x == "Inclination i (degrees)" else None,
             label=f"{ncomponents_labels[ii]} KD component{'s' if ii > 1 else ''}" + r" ($N = %d$)" % (df_SF_subset.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if cc > 0 else None
# axs[0].legend(fontsize="x-small", loc="upper right")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle("Systematics (SF spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_KDcomponents_systematic_SF_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

    
# Specifically looking at v_grad
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)
for cc, col_x in enumerate(["v_grad (component 0)", "v_grad (component 1)", "v_grad (component 2)"]):
    for ii in range(3):
        cond = df_SF_subset["Number of components"] == ii + 1
        if all(df_SF_subset.loc[cond, col_x].isna()):
            continue
        hist(df_SF_subset.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc],
             bins="scott", range=(0, 150),
             label=f"{ncomponents_labels[ii]} KD component{'s' if ii > 1 else ''}" + r" ($N = %d$)" % (df_SF_subset.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x) + f" (component {cc + 1})")
    axs[cc].set_yticklabels([]) if cc > 0 else None
    axs[cc].autoscale(enable=True, axis="x", tight=True)
# axs[0].legend(fontsize="x-small", loc="upper left")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle(r"$v_{\rm grad}$ (SF spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_KDcomponents_vgrad_SF_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

# Local quantities
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)
for cc, col_x in enumerate(["D4000", "r/R_e", "sigma_*"]):
    for ii in range(3):
        cond = df_SF_subset["Number of components"] == ii + 1
        hist(df_SF_subset.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc], 
             range=(vmin_fn(col_x), 100 if col_x.startswith("sigma_gas") else vmax_fn(col_x)),
             bins="scott",
             label=f"{ncomponents_labels[ii]} KD component{'s' if ii > 1 else ''}" + r" ($N = %d$)" % (df_SF_subset.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if cc > 0 else None
    axs[cc].autoscale(enable=True, axis="x", tight=True)
# axs[-1].legend(fontsize="x-small", loc="upper right")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle("Local properties (SF spaxels only)", y=0.94)

# Emission line ratios
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)
for cc, col_x in enumerate(["log O3 (total)", "log N2 (total)", "log S2 (total)"]):
    for ii in range(3):
        cond = df_SF_subset["Number of components"] == ii + 1
        hist(df_SF_subset.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc], 
             range=(vmin_fn(col_x), 100 if col_x.startswith("sigma_gas") else vmax_fn(col_x)),
             bins="scott",
             label=f"{ncomponents_labels[ii]} KD component{'s' if ii > 1 else ''}" + r" ($N = %d$)" % (df_SF_subset.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if cc > 0 else None
    axs[cc].autoscale(enable=True, axis="x", tight=True)
# axs[-1].legend(fontsize="x-small", loc="upper right")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle("Emission line ratios (SF spaxels only)", y=0.94)

# Emission line ratios (metallicity)
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)
for cc, col_x in enumerate(["N2O2 (total)", "N2S2 (total)", "R23 (total)"]):
    for ii in range(3):
        cond = df_SF_subset["Number of components"] == ii + 1
        hist(df_SF_subset.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc], 
             range=(vmin_fn(col_x), 100 if col_x.startswith("sigma_gas") else vmax_fn(col_x)),
             bins="scott",
             label=f"{ncomponents_labels[ii]} KD component{'s' if ii > 1 else ''}" + r" ($N = %d$)" % (df_SF_subset.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if cc > 0 else None
    axs[cc].autoscale(enable=True, axis="x", tight=True)
# axs[-1].legend(fontsize="x-small", loc="upper right")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle("Metallicity-sensitive ratios (SF spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_KDcomponents_local_SF_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

# Global quantities
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
fig.subplots_adjust(wspace=0)
for cc, col_x in enumerate(["R_e (kpc)", "mstar", "log(M/R_e)"]):
    for ii in range(3):
        cond = df_SF_subset["Number of components"] == ii + 1
        hist(df_SF_subset.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc], range=(vmin_fn(col_x), 100 if col_x.startswith("sigma") else vmax_fn(col_x)),
             bins="scott",
             label=f"{ncomponents_labels[ii]} KD component{'s' if ii  > 1 else ''}" + r" ($N = %d$)" % (df_SF_subset.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if cc > 0 else None
    axs[cc].autoscale(enable=True, axis="x", tight=True)
# axs[0].legend(fontsize="x-small", loc="upper left")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle("Global properties (SF spaxels only)", y=0.94)

if savefigs:
    fname = os.path.join(fig_path, f"hist_KDcomponents_global_SF_only.pdf")
    print(f"Saving to {fname}")
    fig.savefig(fname, bbox_inches="tight", format="pdf")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/matplotlib/axes/_axes.py:6735: RuntimeWarning: All-NaN slice encountered
  xmin = min(xmin, np.nanmin(xi))
/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/matplotlib/axes/_axes.py:6736: RuntimeWarning: All-NaN slice encountered
  xmax = max(xmax, np.nanmax(xi))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
# Emission line ratios (metallicity)
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(20, 4))
fig.subplots_adjust(wspace=0)
for cc, col_x in enumerate(["N2O2 (total)", "N2S2 (total)", "R23 (total)", "O3O2 (total)"]):
    for ii in range(3):
        cond = df_SF_subset["Number of components"] == ii + 1
        hist(df_SF_subset.loc[cond, col_x], density=True, histtype="step",
             ax=axs[cc], 
             range=(vmin_fn(col_x), vmax_fn(col_x)),
             bins="scott",
             label=f"{ncomponents_labels[ii]} KD component{'s' if ii > 1 else ''}" + r" ($N = %d$)" % (df_SF_subset.loc[cond].shape[0]),
             color=ncomponents_colours[ii + 1])
    axs[cc].set_xlabel(label_fn(col_x))
    axs[cc].set_yticklabels([]) if cc > 0 else None
    axs[cc].autoscale(enable=True, axis="x", tight=True)
# axs[-1].legend(fontsize="x-small", loc="upper right")
axs[0].set_ylabel(r"$N$ (normalised)")
fig.suptitle("Metallicity-sensitive ratios (SF spaxels only)", y=0.94)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.94, 'Metallicity-sensitive ratios (SF spaxels only)')

In [60]:
df["HALPHA extinction correction"]

0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
             ...   
2817601    5.797336
2817627    1.717270
2823893    2.320312
2826330    1.000000
2826616    2.119703
Name: HALPHA extinction correction, Length: 1840155, dtype: float64

## Hypothesis: low-metallicity regions are more likely to exhibit winds
---

In [67]:
# Compute how many 1, 2, 3 component spaxels there are in bins of SFR surface density (and SFR)
for col_x in ["log N2 (total)", "log O3 (total)", "N2O2 (total)", "O3O2 (total)", "HALPHA extinction correction", "D4000"]:
    sfr_vals = np.linspace(vmin_fn(col_x), vmax_fn(col_x), 11)
    counts_1 = np.zeros(len(sfr_vals) - 1)
    counts_2 = np.zeros(len(sfr_vals) - 1)
    counts_3 = np.zeros(len(sfr_vals) - 1)
    counts_tot = np.zeros(len(sfr_vals) - 1)

    for ll in range(len(sfr_vals) - 1):
        cond = df_SF[col_x] > sfr_vals[ll]
        cond &= df_SF[col_x] <= sfr_vals[ll + 1]
        df_subset = df_SF[cond]
        counts_tot[ll] = df_subset.shape[0]
        counts_1[ll] = df_subset[df_subset["Number of components"] == 1].shape[0]
        counts_2[ll] = df_subset[df_subset["Number of components"] == 2].shape[0]
        counts_3[ll] = df_subset[df_subset["Number of components"] == 3].shape[0]

    # Plot
    fig, axs = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(7, 8))
    fig.subplots_adjust(hspace=0)

    # Plot
    axs[0].bar(sfr_vals[:-1], counts_1,
               align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[1],
               label="1 components")
    axs[0].bar(sfr_vals[:-1], counts_2, bottom=counts_1,
               align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[2],
               label="2 components")
    axs[0].bar(sfr_vals[:-1], counts_3, bottom=counts_1 + counts_2,
               align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[3],
               label="3 components")
    axs[0].grid()
    axs[0].set_ylabel(r"$N$")
    axs[0].set_yscale("log")
    axs[0].autoscale(axis="x", enable=True, tight=True)
    axs[0].legend(loc="upper right", fontsize="small")
    axs[0].set_ylim([0.5, None])
    
    axs[1].bar(sfr_vals[:-1], counts_1 / counts_tot  * 100,
           align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[1])
    axs[1].bar(sfr_vals[:-1], counts_2 / counts_tot  * 100, bottom=counts_1 / counts_tot * 100,
           align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[2])
    axs[1].bar(sfr_vals[:-1], counts_3 / counts_tot * 100, bottom=counts_1 / counts_tot * 100 + counts_2 / counts_tot * 100,
           align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[3])
    axs[1].grid()
    axs[1].set_ylabel("Percentage")
    axs[1].autoscale(axis="x", enable=True, tight=True)
    axs[1].autoscale(axis="y", enable=True, tight=True)
    axs[1].set_xlabel(label_fn(col_x))

    # Heckman+2002 line
    if col_x == "log SFR surface density":
        axs[0].axvline(-1, color="k", linestyle="--")
        axs[1].axvline(-1, color="k", linestyle="--")
        
    if savefigs:
        fname = os.path.join(fig_path, f"hist_SF_only_{col_x.replace(' ', '_')}_ncomponents.pdf")
        print(f"Saving to {fname}")
        fig.savefig(fname, bbox_inches="tight")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in true_divide
/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in true_divide
/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in true_divide


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [76]:
# Compute how many 1, 2, 3 component spaxels there are in bins of SFR surface density (and SFR)
for col_x in ["HALPHA S/N (component 0)", "HALPHA S/N (component 1)", "HALPHA S/N (component 2)", "HALPHA S/N (total)"]:
    sfr_vals = np.linspace(vmin_fn(col_x), vmax_fn(col_x), 11)
    counts_1 = np.zeros(len(sfr_vals) - 1)
    counts_2 = np.zeros(len(sfr_vals) - 1)
    counts_3 = np.zeros(len(sfr_vals) - 1)
    counts_tot = np.zeros(len(sfr_vals) - 1)

    for ll in range(len(sfr_vals) - 1):
        cond = df_SF[col_x] > sfr_vals[ll]
        cond &= df_SF[col_x] <= sfr_vals[ll + 1]
        df_subset = df_SF[cond]
        counts_tot[ll] = df_subset.shape[0]
        counts_1[ll] = df_subset[df_subset["Number of components"] == 1].shape[0]
        counts_2[ll] = df_subset[df_subset["Number of components"] == 2].shape[0]
        counts_3[ll] = df_subset[df_subset["Number of components"] == 3].shape[0]

    # Plot
    fig, axs = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(7, 8))
    fig.subplots_adjust(hspace=0)

    # Plot
    axs[0].bar(sfr_vals[:-1], counts_1,
               align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[1],
               label="1 components")
    axs[0].bar(sfr_vals[:-1], counts_2, bottom=counts_1,
               align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[2],
               label="2 components")
    axs[0].bar(sfr_vals[:-1], counts_3, bottom=counts_1 + counts_2,
               align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[3],
               label="3 components")
    axs[0].grid()
    axs[0].set_ylabel(r"$N$")
    axs[0].set_yscale("log")
    axs[0].autoscale(axis="x", enable=True, tight=True)
    axs[0].legend(loc="upper right", fontsize="small")
    axs[0].set_ylim([0.5, None])
    
    axs[1].bar(sfr_vals[:-1], counts_1 / counts_tot  * 100,
           align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[1])
    axs[1].bar(sfr_vals[:-1], counts_2 / counts_tot  * 100, bottom=counts_1 / counts_tot * 100,
           align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[2])
    axs[1].bar(sfr_vals[:-1], counts_3 / counts_tot * 100, bottom=counts_1 / counts_tot * 100 + counts_2 / counts_tot * 100,
           align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[3])
    axs[1].grid()
    axs[1].set_ylabel("Percentage")
    axs[1].autoscale(axis="x", enable=True, tight=True)
    axs[1].autoscale(axis="y", enable=True, tight=True)
    axs[1].set_xlabel(label_fn(col_x))

    # Heckman+2002 line
    if col_x == "log SFR surface density":
        axs[0].axvline(-1, color="k", linestyle="--")
        axs[1].axvline(-1, color="k", linestyle="--")
        
    if savefigs:
        fname = os.path.join(fig_path, f"hist_SF_only_{col_x.replace(' ', '_')}_ncomponents.pdf")
        print(f"Saving to {fname}")
        fig.savefig(fname, bbox_inches="tight")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## What drives the drop in EW as a function of delta sigma in SF galaxies?
---

In [73]:
col_z_list = ["count"]
for col_z in col_z_list:
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(5 * 3, 5))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])

    for ii in range(3):
        plot2dhistcontours(df_SF, col_x=f"log HALPHA continuum luminosity",
                           col_y=f"log HALPHA EW (component {ii})",
                           col_z=f"{col_z} (component {ii})" if f"{col_z} (component {ii})" in df_SF else col_z, 
                           log_z=True if col_z == "count" else False,
                           alpha=1.0, ax=axs[ii], cax=cax, nbins=100,
                           linewidths=0.5,
                           contours=True, hist=True, colors="white",
                           vmin=1 if col_z == "count" else None, 
                           vmax=1e3 if col_z == "count" else None,
                           plot_colorbar=True if ii == 3 - 1 else False)
        # Decorations
        axs[ii].grid()
        axs[ii].set_ylabel("") if ii > 0 else None
        axs[ii].set_yticklabels([]) if ii > 0 else None
        axs[ii].text(s=f"Component {ii + 1}", x=0.05, y=0.95, transform=axs[ii].transAxes, verticalalignment="top")
   
col_z_list = ["log HALPHA continuum luminosity"]
for col_z in col_z_list:
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(5 * 3, 5))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])

    for ii in range(3):
        plot2dhistcontours(df=df, 
                           col_x=f"log HALPHA luminosity (component {ii})",
                           col_y=f"log HALPHA EW (component {ii})",
                           col_z="count", log_z=True,
                           alpha=0.5, cmap="gray_r",
                           ax=axs[ii], plot_colorbar=False)

        plot2dhistcontours(df_SF, col_x=f"log HALPHA luminosity (component {ii})",
                           col_y=f"log HALPHA EW (component {ii})",
                           col_z=f"{col_z} (component {ii})" if f"{col_z} (component {ii})" in df_SF else col_z, 
                           log_z=True if col_z == "count" else False,
                           alpha=1.0, ax=axs[ii], cax=cax, nbins=100,
                           linewidths=0.5,
                           contours=True, hist=True, colors="white",
                           vmin=1 if col_z == "count" else None, 
                           vmax=1e3 if col_z == "count" else None,
                           plot_colorbar=True if ii == 3 - 1 else False)
        # Decorations
        axs[ii].grid()
        axs[ii].set_ylabel("") if ii > 0 else None
        axs[ii].set_yticklabels([]) if ii > 0 else None
        axs[ii].text(s=f"Component {ii + 1}", x=0.05, y=0.95, transform=axs[ii].transAxes, verticalalignment="top")
   
       

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [74]:
plt.close("all")